In [1]:
from aocd import data, models, submit
from io import StringIO
from pathlib import Path
import re

import pandas as pd
from collections import defaultdict, namedtuple

# Load data and examples

In [2]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [3]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
todays_examples = todays_puzzle.examples
data = todays_puzzle.input_data

In [4]:
def is_full_interval(matches_sorted_on_first_match: defaultdict, interval: tuple):
    min_value, max_value = interval
    sorted_keys = sorted(matches_sorted_on_first_match.keys())
    if min_value in sorted_keys:
        for match_span in matches_sorted_on_first_match[min_value]:
            if match_span[1] == max_value:
                return True
            if is_full_interval(
                matches_sorted_on_first_match, (match_span[1], max_value)
            ):
                return True
    return False

In [5]:
def is_patternable(towels: list, pattern: str) -> bool:
    matches_sorted_on_first_match = defaultdict(list)
    for towel in towels:
        for match in re.finditer(
            towel, pattern
        ):  # this is non-overalapping match. It works in part a, but does not work in part b
            i, j = match.span()
            matches_sorted_on_first_match[i].append((i, j))
    len(matches_sorted_on_first_match)
    return is_full_interval(matches_sorted_on_first_match, (0, len(pattern)))

# Part A

In [6]:
def part_a(data: str) -> str:
    towels = data.splitlines()[0].split(", ")
    patterns = data.splitlines()[2:]
    result = 0
    for pattern in patterns:
        if is_patternable(towels, pattern):
            result += 1
    return str(result)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        print(
            f"Example {example_index} part a: {part_a(example.input_data)} (expected {example.answer_a})"
        )
        assert part_a(str(example.input_data)) == example.answer_a
submit(part_a(data), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [9]:
from tqdm import tqdm
from functools import lru_cache

In [10]:
TowelMatch = namedtuple("Match", ["start", "end", "pattern"])

In [107]:
def matching_patterns(
    matches_sorted_on_first_match: defaultdict, interval: tuple
) -> int:
    sorted_keys = sorted(matches_sorted_on_first_match.keys())

    @lru_cache
    def matching_patterns_inner(interval: tuple):
        min_value, max_value = interval
        result = 0
        if min_value in sorted_keys:
            for i, towel_match in enumerate(matches_sorted_on_first_match[min_value]):
                if towel_match.end == max_value:
                    result += 1
                else:
                    count_sub_patterns = matching_patterns_inner(
                        (towel_match.end, max_value)
                    )
                    result += count_sub_patterns
        return result

    result = matching_patterns_inner(interval)
    return result

In [108]:
def count_different_matching_combinations(towels: list, pattern: str) -> bool:
    matches_sorted_on_first_match = defaultdict(list)
    for towel in towels:
        for match in re.finditer(f"(?=({towel}))", pattern):
            i, j = match.span(1)
            matches_sorted_on_first_match[i].append(TowelMatch(i, j, towel))
    count_patterns = matching_patterns(matches_sorted_on_first_match, (0, len(pattern)))
    return count_patterns

In [109]:
def part_b(data: str) -> str:
    towels = data.splitlines()[0].split(", ")
    patterns = data.splitlines()[2:]
    result = 0
    count_designs = 0
    for pattern in tqdm(patterns):
        old_result = result
        result += count_different_matching_combinations(towels, pattern)
        count_designs += 1
    return str(result)

In [111]:
todays_examples[0] = todays_examples[0]._replace(answer_b="16")

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_b != "":
        print(
            f"Example {example_index} part b: {part_b(example.input_data)} (expected {example.answer_b})"
        )
        assert part_b(str(example.input_data)) == example.answer_b
submit(part_b(data), part="b", year=puzzle_year, day=puzzle_day)